# Global Imports

In [ ]:
import pprint
from pathlib import Path as pt

import numpy as np

from utils_002 import Gen_XYZ, gen_xyz

# Functions

In [ ]:
def Gen_XYZ(
    size_x: int = 10, size_y: int = 10, size_z: int = 10, mode: int = 1
) -> tuple[np.ndarray, np.ndarray, np.ndarray] | tuple[None, None, None]:
    """
    Generates 3D coordinates (X, Y, Z) based on specified dimensions and a masking mode.

    Args:
        size_x (int): Dimension along the X-axis. Defaults to 10.
        size_y (int): Dimension along the Y-axis. Defaults to 10.
        size_z (int): Dimension along the Z-axis. Defaults to 10.
        mode (int): The masking mode to apply:
            1: Returns coordinates where (X + Y + Z) is even (checkerboard pattern).
            2: Returns all coordinates within the specified bounds (no effective mask).
            Any other value: Returns (None, None, None) and prints an error.

    Returns:
        tuple[np.ndarray, np.ndarray, np.ndarray] | tuple[None, None, None]:
            A tuple containing three NumPy arrays (X, Y, Z) of masked coordinates,
            or (None, None, None) if an invalid mode is provided or an error occurs.
    """
    if not all(isinstance(dim, int) and dim > 0 for dim in [size_x, size_y, size_z]):
        print("Error: Dimensions (size_x, size_y, size_z) must be positive integers.")
        return None, None, None

    try:
        X_full, Y_full, Z_full = np.mgrid[0:size_x, 0:size_y, 0:size_z]

        if mode == 1:
            mask = (X_full + Y_full + Z_full) % 2 == 0
        elif mode == 2:
            mask = np.ones_like(X_full).astype(bool)
        else:
            print(f"Error: Invalid mode specified. Expected 1 or 2, but got {mode}.")
            return None, None, None

        X = X_full[mask].flatten()
        Y = Y_full[mask].flatten()
        Z = Z_full[mask].flatten()

        return X, Y, Z

    except MemoryError:
        print(
            f"Error: Not enough memory to create grid of size {size_x}x{size_y}x{size_z}."
        )
        return None, None, None
    except Exception as e:
        print(f"An unexpected error occurred during coordinate generation: {e}")
        return None, None, None

In [ ]:
def prep_cuboid(
    state: np.ndarray,
    XYZ: tuple[np.ndarray, np.ndarray, np.ndarray],
    start_point: tuple[float, float, float] = (0.0, 0.0, 0.0),
    sizes: tuple[float, float, float] = (1.0, 1.0, 1.0),
    value: int = 1,
) -> np.ndarray:
    """
    Create a 3D cuboid mask within a grid defined by XYZ coordinates.

    Args:
        XYZ: Tuple of 1D arrays (X, Y, Z) representing grid coordinates.
        start_point: Tuple (x0, y0, z0) defining the cuboid's starting corner (default: (0, 0, 0)).
        sizes: Tuple (sx, sy, sz) defining the cuboid's dimensions (default: (1, 1, 1)).
        value: Value to assign to the cuboid region (default: 1).
        dtype: Data type of the output array (default: np.int32).

    Returns:
        np.ndarray: 1D array of the same size as X.size, with `value` in the cuboid region and 0 elsewhere.

    Raises:
        ValueError: If inputs are invalid (e.g., negative sizes, mismatched grid sizes).
        TypeError: If input types are incorrect.
    """
    # Input validation
    if not isinstance(XYZ, tuple) or len(XYZ) != 3:
        raise TypeError("XYZ must be a tuple of three numpy arrays.")
    X, Y, Z = XYZ
    if not all(isinstance(arr, np.ndarray) and arr.ndim == 1 for arr in (X, Y, Z)):
        raise TypeError("X, Y, Z must be 1D numpy arrays.")
    if not (X.size == Y.size == Z.size):
        raise ValueError("X, Y, Z must have the same size.")
    if not isinstance(start_point, tuple) or len(start_point) != 3:
        raise TypeError("start_point must be a tuple of three numbers.")
    if not isinstance(sizes, tuple) or len(sizes) != 3:
        raise TypeError("sizes must be a tuple of three numbers.")

    # Ensure sizes are positive
    sizes = tuple(max(float(s), 1.0) for s in sizes)  # Avoid zero/negative sizes

    # Compute cuboid mask using vectorized operations
    mask = (
        (X >= start_point[0])
        & (X < start_point[0] + sizes[0])
        & (Y >= start_point[1])
        & (Y < start_point[1] + sizes[1])
        & (Z >= start_point[2])
        & (Z < start_point[2] + sizes[2])
    )

    # Assign value to cuboid region
    state[mask] = value

    return state

# Preparation of the InitStates file

In [ ]:
np.random.seed(1012)

Sx, Sy, Sz = 50, 50, 50
X, Y, Z = gen_xyz(Sx, Sy, Sz, mode=2)
size = X.size

state = np.zeros(X.size, dtype=int)

state = prep_cuboid(
    state, XYZ=(X, Y, Z), start_point=(0, 0, 0), sizes=(Sx // 2, Sy, Sz)
)  # Filled 1/0 50/50 % by X axis

# state = prep_cuboid(
#     state, XYZ=(X, Y, Z), start_point=(0, 0, 0), sizes=(Sx, Sy // 2, Sz)
# )  # Filled 1/0 50/50 % by Y axis

# state = prep_cuboid(
#     state, XYZ=(X, Y, Z), start_point=(0, 0, 0), sizes=(Sx, Sy, Sz // 2)
# )  # Filled 1/0 50/50 % by X axis

# CuboidX, CuboidY, CuboidZ = 10, 10, 10
# state = prep_cuboid(
#     state,
#     XYZ=(X, Y, Z),
#     start_point=(
#         (Sx - CuboidX) // 2,
#         (Sy - CuboidY) // 2,
#         (Sz - CuboidZ) // 2,
#     ),
#     sizes=(CuboidX, CuboidY, CuboidZ),
# )  # Filled Cuboid in Grid center

mask = X == (Sx // 2)
# mask = Y == (Sy // 2)
# mask = Z == (Sz // 2)
state[mask] = np.random.randint(low=0, high=1 + 1, size=np.count_nonzero(mask))

print(np.unique(state, return_counts=True))

# pt("InitStates.ini").open(mode="w", encoding="utf-8").write(
#     ":".join(["%d"] * size) % tuple(state.tolist())
# )
pt("InitStates.ini").write_text(
    ":".join(["%d"] * size) % tuple(state.tolist()), encoding="utf-8"
)

# Preparation of the Combinations file

In [ ]:
"""Зміст файлу ``InitSettings.ini``"""

InitSettings = pt("InitSettings.ini").read_text(encoding="utf-8", errors="ignore")
print(InitSettings)

In [ ]:
template_ini = """
DirPrefix:{}
     Seed:{}

       Sx:{}
       Sy:{}
       Sz:{}
       Px:{}
       Py:{}
       Pz:{}

        T:{:.5f}
       Ax:{:.5e}
       Ay:{:.5e}
       Az:{:.5e}

     g100:{:.5f}
     g010:{:.5f}
     g001:{:.5f}

     mode:{}
       dg:{:.5e}
     C_eq:{:.5e}
       C0:{:.5e}
    N_tot:{:.5e}
    N0_cr:{:.5e}
      p_b:{:.5f}

     AddI:{}
  AddFrom:{}
     RemI:{}
  RemFrom:{}

 LoadPrev:{}

  StepLim:{}
   PrintI:{}
   WriteI:{}

/////////////////////// | Для коментарів | /////////////////////////
Version: {}
ExeFileName: {}

DirPrefix     — Назва папки для результатів
Seed          — Початкове зерно для PRNG

Sx, Sy, Sz    — Розміри ґратки по X, Y, Z
Px, Py, Pz    — Періодичність по X, Y, Z (true / false)

T             — Температура (K)
Ax, Ay, Az    — Константи решітки по X, Y, Z (м)
g100, g010,
g001          — Анізотропна поверхнева енергія (напрямки [100], [010], [001])

mode          — Режим симуляції (1 = звичайна, інші TBD)
dg            — Δμ, різниця хімпотенціалів
C_eq          — Рівноважна концентрація
C0            — Початкова концентрація
N_tot         — Загальна кількість атомів
N0_cr         — Початкове число атомів у кристалі
p_b           — Ймовірність баллістичного переміщення

AddI, AddFrom — Частота та старт додавання
RemI, RemFrom — Частота та старт видалення

LoadPrev      — Завантаження стану:
                "0"    - не завантажувати
                "-1"   - останній стан
                "N"    - завантажити N-й

StepLim       — Максимальна кількість кроків
PrintI        — Інтервал виводу в консоль
WriteI        — Інтервал запису у файл

//////////////////// | Додаткові коментарі | ///////////////////////
{}
"""

template_comb = " +| ".join(
    (
        "{} +| {}",
        "{} +| {} +| {} +| {} +| {} +| {}",
        "{:.5f} +| {:.5e} +| {:.5e} +| {:.5e}",
        "{:.5f} +| {:.5f} +| {:.5f}",
        "{} +| {:.5e} +| {:.5e} +| {:.5e} +| {:.5e} +| {:.5e} +| {:.5f}",
        "{} +| {} +| {} +| {}",
        "{}",
        "{} +| {} +| {}",
        "{} +| {}",
        "{}",
    )
)

In [ ]:
def DefaultInitSettings():
    return {
        "DirPrefix": "Base",
        "Seed": 1012,
        "Sx": 51,
        "Sy": 51,
        "Sz": 51,
        "Px": False,
        "Py": False,
        "Pz": False,
        "T": 300.0,
        "Ax": 5.85e-10,
        "Ay": 1.78e-10,
        "Az": 4.41e-10,
        "g100": 0.41,
        "g010": 0.54,
        "g001": 0.22,
        "mode": 1,
        "dg": 0.0,
        "C_eq": 9.58767e-08,
        "C0": 9.58767e-08,
        "N_tot": 5e12,
        "N0_cr": -1.0,
        "p_b": -1.0,
        "AddI": 1,
        "AddFrom": 1,
        "RemI": 1,
        "RemFrom": 1,
        "LoadPrev": 0,
        "StepLim": 1_000_000,
        "PrintI": 100_000,
        "WriteI": 10_000,
        "Version": "6.0.0",
        "ExeFileName": "M1_v6.0.0.exe",
        "comments_adds": "",
    }

In [ ]:
Ax = 5.85E-10
Ay = 1.78E-10
Az = 4.41E-10

# A1x = Ax / (2 ** (1 / 3))
# A1y = Ay / (2 ** (1 / 3))
# A1z = Az / (2 ** (1 / 3))

# A1x = Ax / (7 ** (1 / 3))
# A1y = Ay / (7 ** (1 / 3))
# A1z = Az / (7 ** (1 / 3))

A1x = Ax / (14 ** (1 / 3))
A1y = Ay / (14 ** (1 / 3))
A1z = Az / (14 ** (1 / 3))

g100 = 0.41
g010 = 0.54
g001 = 0.22

Ω = A1x * A1y * A1z

EX = 2 * g100 * A1y * A1z
EY = 2 * g010 * A1z * A1x
EZ = 2 * g001 * A1x * A1y

Eisol = EX + EY + EZ
E3 = EY + EX
E2 = EZ + EY
E1 = EZ + EX

kB = 1.380649e-23
T = 300.0
kT = kB * T
Ceq = np.exp(- Eisol / kT)

print(
    """
Sizes:
    Ax: {:.5e},    Ay: {:.5e},    Az: {:.5e}
   A1x: {:.5e},   A1y: {:.5e},   A1z: {:.5e}

Volume:
   Ω = A1x * A1y * A1z = {:.5e} * {:.5e} * {:.5e} = {:.5e}

Surface tensions:
   g100: {:.5e}, g010: {:.5e}, g001: {:.5e}

Energys:
1)
   EX = 2.0 * g100 * A1y * A1z = 2.0 * {:.5e} * {:.5e} * {:.5e} = {:.5e}
   EY = 2.0 * g010 * A1z * A1x = 2.0 * {:.5e} * {:.5e} * {:.5e} = {:.5e}
   EZ = 2.0 * g001 * A1x * A1y = 2.0 * {:.5e} * {:.5e} * {:.5e} = {:.5e}

2)
   Eisol = EX + EY + EZ = {:.5e} + {:.5e} + {:.5e} = {:.5e}

3)
   E3 = EY + EX = {:.5e} + {:.5e} = {:.5e}
   E2 = EZ + EY = {:.5e} + {:.5e} = {:.5e}
   E1 = EZ + EX = {:.5e} + {:.5e} = {:.5e}

kB = {:.5e}; T = {:.5f};
kT = kB * T = {:.5e} * {:.5f} = {:.5e};
Ceq = exp(- Eisol / kT) = exp(- {:.5e} / {:.5e} ) = {:.5e};
""".format(
        Ax, Ay, Az,
        A1x, A1y, A1z,
        A1x, A1y, A1z,
        Ω,
        g100, g010, g001,
        g100, A1y, A1z, EX,
        g010, A1z, A1x, EY,
        g001, A1x, A1y, EZ,
        EX, EY, EZ, Eisol,
        EY, EX, E3,
        EZ, EY, E2,
        EZ, EX, E1,
        kB, T,
        kB, T, kT,
        Eisol, kT, Ceq,
    ).strip()
)

In [ ]:
range_01_0_to_E1 = np.linspace(0.0, E1, 11)
range_01_E1_to_E2 = np.linspace(E1, E2, 11)
range_01_E2_to_E3 = np.linspace(E2, E3, 11)
range_01_E3_to_Eisol = np.linspace(E3, Eisol, 11)

dg_s_positive = np.hstack(
    [
        # range_01_0_to_E1,
        # range_01_E1_to_E2,
        # range_01_E2_to_E3,
        # range_01_E3_to_Eisol,
        # ======================
        range_01_0_to_E1,
        range_01_E1_to_E2[1:],
        range_01_E2_to_E3[1:],
        range_01_E3_to_Eisol[1:],
    ]
)

dg_s_negative = (-1.0 * dg_s_positive)[::-1]
dg_s_negative[np.abs(dg_s_negative) <= 1e-32] = 0.0

dg_s_full = np.hstack(
    [
        dg_s_negative,
        dg_s_positive[1:],
    ]
)

# dg_s = np.arange(0.0, Eisol + 1e-21, 1e-21)

dg_s = dg_s_positive
# dg_s = dg_s_negative
# dg_s = dg_s_full

dg_s = np.around(dg_s, 32)
dg_s

In [ ]:
InitSettings = DefaultInitSettings()
pprint.pp(InitSettings, indent=2)

In [ ]:
InitSettings["Version"] = "6.0.0"
InitSettings["ExeFileName"] = "M1_v6.0.0.exe"

InitSettings["DirPrefix"] = "Rn1"

InitSettings["Sx"] = 51
InitSettings["Sy"] = 51
InitSettings["Sz"] = 51
InitSettings["Px"] = False
InitSettings["Py"] = False
InitSettings["Pz"] = False

InitSettings["Ax"] = A1x
InitSettings["Ay"] = A1y
InitSettings["Az"] = A1z

InitSettings["LoadPrev"] = 0

InitSettings["StepLim"] = 1_000_000
InitSettings["PrintI"] = 100_000
InitSettings["WriteI"] = 10_000

comments_adds_t = """
Ax = 5.85E-10 / (14 ^ (1 / 3))
Ay = 1.78E-10 / (14 ^ (1 / 3))
Az = 4.41E-10 / (14 ^ (1 / 3))
""".strip()

prepared_combs_lines = []
for dg in dg_s:
    InitSettings["dg"] = dg
    InitSettings["comments_adds"] = comments_adds_t

    comb_str = template_comb.format(
        InitSettings["DirPrefix"],
        InitSettings["Seed"],
        # Logic Block SPLITTER
        InitSettings["Sx"],
        InitSettings["Sy"],
        InitSettings["Sz"],
        str(InitSettings["Px"]).lower(),
        str(InitSettings["Py"]).lower(),
        str(InitSettings["Pz"]).lower(),
        # Logic Block SPLITTER
        InitSettings["T"],
        InitSettings["Ax"],
        InitSettings["Ay"],
        InitSettings["Az"],
        # Logic Block SPLITTER
        InitSettings["g100"],
        InitSettings["g010"],
        InitSettings["g001"],
        # Logic Block SPLITTER
        InitSettings["mode"],
        InitSettings["dg"],
        InitSettings["C_eq"],
        InitSettings["C0"],
        InitSettings["N_tot"],
        InitSettings["N0_cr"],
        InitSettings["p_b"],
        # Logic Block SPLITTER
        InitSettings["AddI"],
        InitSettings["AddFrom"],
        InitSettings["RemI"],
        InitSettings["RemFrom"],
        # Logic Block SPLITTER
        InitSettings["LoadPrev"],
        # Logic Block SPLITTER
        InitSettings["StepLim"],
        InitSettings["PrintI"],
        InitSettings["WriteI"],
        # Logic Block SPLITTER
        InitSettings["Version"],
        InitSettings["ExeFileName"],
        # Logic Block SPLITTER
        InitSettings["comments_adds"].strip().replace("\n", "\\n\\"),
    ).strip()

    prepared_combs_lines.append(comb_str)

with open("WorkCombs.txt", mode="w", encoding="utf-8") as ff:
    ff.write("\n".join(prepared_combs_lines).strip())

In [ ]:
import os
import random as rnd
from dataclasses import dataclass
from distutils.util import strtobool
from pathlib import Path
from subprocess import CREATE_NEW_CONSOLE, Popen
from time import sleep
from typing import Dict, List, Optional, Set, Tuple, Union

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pyvista as pv
from matplotlib import gridspec
from numpy import random as nrnd


def custom_strtobool(val: str) -> bool:
    """Converts a string to a boolean value."""
    val = val.lower()
    if val in ("y", "yes", "t", "true", "on", "1"):
        return True
    elif val in ("n", "no", "f", "false", "off", "0"):
        return False
    else:
        raise ValueError(f"Недопустимое булево значение: {val}")


def load_Cfg(
    path_to_config,
    keys_to_bool: set = None,
    keys_to_int: set = None,
    keys_to_float: set = None,
    keys_to_str: set = None,
    com_line: str = "///////////////////////// | Для коментарів | /////////////////////////",
) -> dict:
    """
    Loads configuration from an INI file using configparser.
    Applies type conversions based on provided key sets.
    """

    _keys_to_bool = (
        keys_to_bool
        if keys_to_bool is not None
        else {"PeriodicX", "PeriodicY", "PeriodicZ"}
    )
    _keys_to_int = (
        keys_to_int
        if keys_to_int is not None
        else {
            "Seed",
            "Sx",
            "Sy",
            "Sz",
            "LoadPrevState",
            "AddInterval",
            "RemInterval",
            "RemStartFrom",
            "GenStepsLimit",
            "PrintInterval",
            "WriteInterval",
        }
    )
    _keys_to_float = (
        keys_to_float
        if keys_to_float is not None
        else {"T", "Ax", "Ay", "Az", "g100", "g010", "g001", "dg"}
    )
    _keys_to_str = (
        keys_to_str
        if keys_to_str is not None
        else {
            "DirPrefix",
        }
    )

    try:
        cfg_dict = [
            *map(
                lambda line: line.strip(),
                open(path_to_config, mode="r", encoding="utf-8")
                .read()
                .strip()
                .split("\n"),
            )
        ]

        com_line_start_id = cfg_dict.index(com_line)

        cfg_part = [*filter(lambda item: item != "", cfg_dict[:com_line_start_id])]
        com_part = cfg_dict[com_line_start_id:]

        cfg_dict = {}

        for line in cfg_part:
            k, v = line.split(":")
            k, v = k.strip(), v.strip()

            if k in _keys_to_int:
                try:
                    cfg_dict[k] = int(v)
                except ValueError:
                    print(
                        f"Ошибка: Не удалось преобразовать значение '{v}' для ключа '{k}' в целое число. Проверьте WorkCfg.ini.",
                        file=sys.stderr,
                    )
                    sys.exit(1)
            elif k in _keys_to_float:
                try:
                    cfg_dict[k] = float(v)
                except ValueError:
                    print(
                        f"Ошибка: Не удалось преобразовать значение '{v}' для ключа '{k}' в число с плавающей точкой. Проверьте WorkCfg.ini.",
                        file=sys.stderr,
                    )
                    sys.exit(1)
            elif k in _keys_to_bool:
                try:
                    cfg_dict[k] = custom_strtobool(v)
                except ValueError:
                    print(
                        f"Ошибка: Не удалось преобразовать значение '{v}' для ключа '{k}' в булево. Проверьте WorkCfg.ini.",
                        file=sys.stderr,
                    )
                    sys.exit(1)
            elif k in _keys_to_str:
                if (v.startswith('"') & v.endswith('"')) | (
                    v.startswith("'") & v.endswith("'")
                ):
                    v = v[1:-1]
                cfg_dict[k] = v
            else:
                cfg_dict[k] = v

        return cfg_dict
    except FileNotFoundError:
        print(
            f"Ошибка: Файл конфигурации '{path_to_config}' не найден.", file=sys.stderr
        )
        sys.exit(1)
    except Exception as e:
        print(
            f"Неизвестная ошибка при загрузке файла конфигурации '{path_to_config}': {e}",
            file=sys.stderr,
        )
        sys.exit(1)


def process_directory_1(dir_path: Path) -> Tuple[Dict, Optional[np.ndarray]]:
    try:
        path_cfg = dir_path / "InitSettings.ini"
        path_states = dir_path / "TimeStates.txt"

        if not (path_cfg.exists() and path_states.exists()):
            raise FileNotFoundError(f"Required files missing in {dir_path}")

        data_ini = load_Cfg(path_to_config=path_cfg)
        run_id = dir_path.name.split("_")[0]
        data_ini["id"] = run_id
        size_x, size_y, size_z = data_ini["Sx"], data_ini["Sy"], data_ini["Sz"]

        time_idx = 0
        with open(path_states, mode="r") as f:
            for line in f:
                line = line.strip()
                if not line:
                    break

                time_idx += 1

        return data_ini["WriteInterval"], data_ini["dg"], time_idx

    except Exception as e:
        print(f"Error processing directory {dir_path}: {str(e)}")
        raise


base_folder = Path(
    r"1749736878475"
)

res = []

try:
    for root, _, files in os.walk(base_folder):
        if {"InitSettings.ini", "TimeStates.txt"}.issubset(files):
            print(f"Processing directory: {root}")
            WriteInterval, dg_value, time_idx = process_directory_1(Path(root))
            res.append([WriteInterval, dg_value, time_idx])
except Exception as e:
    print(f"Error in main loop: {str(e)}")
    raise

In [ ]:
res = np.array(res).T
res

In [ ]:
fig, ax = plt.subplots(1, figsize=(8, 4), dpi=300)
ax.plot(
    res[1],
    res[2],
    color="green",
    linewidth=1,
    marker="o",
    markersize=2,
    markeredgewidth=0.5,
    markerfacecolor="blue",
    markeredgecolor="black",
    markevery=1,
    label="",
)

# ax.set_title(f"Axis X")
# ax.set_title(f"Axis Y")
ax.set_title(f"Axis Z")
# ax.legend(loc="best")
ax.set_xlabel("Value of dg")
ax.set_ylabel("Mean velocity of the crystal/liquid interface")
ax.grid(True)
# plt.savefig(Path(r"AxisVelocity") / f"AxisX.png")
plt.show()
plt.close()

In [ ]:
fig, ax = plt.subplots(1, figsize=(8, 4), dpi=300)
ax.plot(
    res[1],
    res[2],
    color="green",
    linewidth=1,
    marker="o",
    markersize=2,
    markeredgewidth=0.5,
    markerfacecolor="blue",
    markeredgecolor="black",
    markevery=1,
    label="",
)

# ax.set_title(f"Axis X")
# ax.set_title(f"Axis Y")
ax.set_title(f"Axis Z")
# ax.legend(loc="best")
ax.set_xlabel("Value of dg")
ax.set_ylabel("Mean velocity of the crystal/liquid interface")
ax.grid(True)
# plt.savefig(Path(r"AxisVelocity") / f"AxisX.png")
plt.show()
plt.close()

In [ ]:
InitSettings["Version"] = "6.0.0"
InitSettings["ExeFileName"] = "M1_v6.0.0.exe"

InitSettings["DirPrefix"] = "AxisZ"

InitSettings["Sx"] = 50
InitSettings["Sy"] = 50
InitSettings["Sz"] = 50
InitSettings["PeriodicX"] = True
InitSettings["PeriodicY"] = True
InitSettings["PeriodicZ"] = True

InitSettings["Ax"] = 5.85e-10 / (7 ** (1 / 3))
InitSettings["Ay"] = 1.78e-10 / (7 ** (1 / 3))
InitSettings["Az"] = 4.41e-10 / (7 ** (1 / 3))

InitSettings["LoadPrevState"] = 1

InitSettings["GenStepsLimit"] = 10_000_000
InitSettings["PrintInterval"] = 500_000
InitSettings["WriteInterval"] = 10_000

comments_adds_t = """
Ax = 5.85E-10 / (7 ^ (1 / 3))
Ay = 1.78E-10 / (7 ^ (1 / 3))
Az = 4.41E-10 / (7 ^ (1 / 3))
""".strip()

prepared_combs_lines = []
for dg, WriteInterval in np.vstack([res[1], (res[0] * (res[2] - 2)) / 100]).T:
    InitSettings["dg"] = dg
    InitSettings["WriteInterval"] = int(WriteInterval)
    InitSettings["comments_adds"] = comments_adds_t

    comb_str = template_comb.format(
        InitSettings["DirPrefix"],
        InitSettings["Seed"],
        # Logic Block SPLITTER
        InitSettings["Sx"],
        InitSettings["Sy"],
        InitSettings["Sz"],
        str(InitSettings["PeriodicX"]).lower(),
        str(InitSettings["PeriodicY"]).lower(),
        str(InitSettings["PeriodicZ"]).lower(),
        # Logic Block SPLITTER
        InitSettings["T"],
        InitSettings["Ax"],
        InitSettings["Ay"],
        InitSettings["Az"],
        InitSettings["g100"],
        InitSettings["g010"],
        InitSettings["g001"],
        InitSettings["dg"],
        InitSettings["LoadPrevState"],
        # Logic Block SPLITTER
        InitSettings["AddInterval"],
        InitSettings["RemInterval"],
        InitSettings["RemStartFrom"],
        # Logic Block SPLITTER
        InitSettings["GenStepsLimit"],
        InitSettings["PrintInterval"],
        InitSettings["WriteInterval"],
        # Logic Block SPLITTER
        InitSettings["Version"],
        InitSettings["ExeFileName"],
        # Logic Block SPLITTER
        InitSettings["comments_adds"].strip().replace("\n", "\\n\\"),
    ).strip()

    prepared_combs_lines.append(comb_str)

with open("WorkCombs.txt", mode="w", encoding="utf-8") as ff:
    ff.write("\n".join(prepared_combs_lines).strip())